In [ ]:
!pip install yfinance 

     |████████████████████████████████| 6.3 MB 2.6 MB/s 
  Created wheel for yfinance: filename=yfinance-0.1.63-py2.py3-none-any.whl size=23918 sha256=2d2c45a35297f0f10f74a9b2723924b40b8899c38f5afe319e20d9c667ba4c6d
  Stored in directory: /root/.cache/pip/wheels/fe/87/8b/7ec24486e001d3926537f5f7801f57a74d181be25b11157983
Successfully built yfinance
  Attempting uninstall: lxml
    Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6


In [ ]:
import yfinance as yf
import numpy as np
import pandas as pd
import tensorflow as tf

In [ ]:
data = yf.download("SBIN.NS" , start = "2018-01-01" , interval = '1d')

[*********************100%***********************]  1 of 1 completed


In [ ]:
data.shape

(912, 6)

In [ ]:
data.head(3)

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2018-01-01,310.600006,312.750000,306.299988,307.100006,304.290649,12182400
2018-01-02,309.000000,309.000000,301.350006,303.250000,300.475861,14797364
2018-01-03,305.299988,306.450012,302.149994,302.850006,300.079529,13857373


Understanding Trends with in the Data

In [ ]:
# Sort the data points based on indexes just for confirmation 
data.sort_index(inplace = True)

In [ ]:
# Remove any duplicate index 
data = data.loc[~data.index.duplicated(keep='first')]

In [ ]:
data.tail(3)

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2021-09-13,430.100006,433.799988,428.799988,432.850006,432.850006,9567921
2021-09-14,435.000000,436.750000,432.250000,433.000000,433.000000,9981562
2021-09-15,433.000000,445.000000,431.450012,443.850006,443.850006,22173628


In [ ]:
# Check for missing values 
data.isnull().sum()

Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64

In [ ]:
# Get the statistics of the data
data.describe()

,Open,High,Low,Close,Adj Close,Volume
count,912.000000,912.000000,912.000000,912.000000,912.000000,9.120000e+02
mean,291.822643,295.945121,287.153070,291.428344,289.070984,3.818661e+07
std,68.668556,69.057350,68.184095,68.672891,68.662382,2.687744e+07
min,151.949997,153.199997,149.449997,150.850006,149.470016,3.312160e+06
25%,252.974998,257.925011,248.399994,253.674999,251.354366,1.895467e+07
50%,288.375000,291.799988,283.574997,287.375000,284.746078,3.048220e+07
75%,331.000000,333.899994,325.025002,329.974998,326.956367,4.963601e+07
max,461.000000,467.450012,441.500000,456.950012,456.950012,2.149557e+08


In [ ]:
import plotly.graph_objects as go

# Check the trend in Closing Values 
fig = go.Figure()

fig.add_trace(go.Scatter(x = data.index , y = data['Close'] , mode = 'lines'))
fig.update_layout(height = 500 , width = 900, 
                  xaxis_title='Date' , yaxis_title='Close')
fig.show()

In [ ]:
# Check the trend in Volume Traded
fig = go.Figure()

fig.add_trace(go.Scatter(x = data.index , y = data['Volume'] , mode = 'lines'))
fig.update_layout(height = 500 , width = 900, 
                  xaxis_title='Date' , yaxis_title='Volume')
fig.show()

Data Preparation

In [ ]:
from sklearn.preprocessing import MinMaxScaler 
import pickle 
from tqdm.notebook import tnrange

In [ ]:
# Filter only required data 
data = data[['Close' , 'Volume']]
data.head(3)

,Close,Volume
Date,,
2018-01-01,307.100006,12182400
2018-01-02,303.250000,14797364
2018-01-03,302.850006,13857373


In [ ]:
# Confirm the Testing Set length 
test_length = data[(data.index >= '2020-09-01')].shape[0]

In [ ]:
def CreateFeatures_and_Targets(data, feature_length):
    X = []
    Y = []

    for i in tnrange(len(data) - feature_length): 
        X.append(data.iloc[i : i + feature_length,:].values)
        Y.append(data["Close"].values[i+feature_length])

    X = np.array(X)
    Y = np.array(Y)

    return X , Y

In [ ]:
X , Y = CreateFeatures_and_Targets(data , 32)

  0%|          | 0/880 [00:00<?, ?it/s]

In [ ]:
# Check the shapes
X.shape , Y.shape

((880, 32, 2), (880,))

In [ ]:
Xtrain , Xtest , Ytrain , Ytest = X[:-test_length] , X[-test_length:] , Y[:-test_length] , Y[-test_length:]

In [ ]:
# Check Training Dataset Shape 
Xtrain.shape , Ytrain.shape

((622, 32, 2), (622,))

In [ ]:
# Check Testing Dataset Shape
Xtest.shape , Ytest.shape

((258, 32, 2), (258,))

In [ ]:
# Create a Scaler to Scale Vectors with Multiple Dimensions 
class MultiDimensionScaler():
    def __init__(self):
        self.scalers = []

    def fit_transform(self , X):
        total_dims = X.shape[2]
        for i in range(total_dims):
            Scaler = MinMaxScaler()
            X[:, :, i] = Scaler.fit_transform(X[:, :, i])
            self.scalers.append(Scaler)
        return X

    def transform(self , X):
        for i in range(X.shape[2]):
            X[:, :, i] = self.scalers[i].transform(X[:,:,i])
        return X 

In [ ]:
Feature_Scaler = MultiDimensionScaler()
Xtrain = Feature_Scaler.fit_transform(Xtrain)
Xtest = Feature_Scaler.transform(Xtest)

In [ ]:
Target_Scaler = MinMaxScaler()
Ytrain = Target_Scaler.fit_transform(Ytrain.reshape(-1,1))
Ytest = Target_Scaler.transform(Ytest.reshape(-1,1))

In [ ]:
def save_object(obj , name : str):
    pickle_out = open(f"{name}.pck","wb")
    pickle.dump(obj, pickle_out)
    pickle_out.close()

def load_object(name : str):
    pickle_in = open(f"{name}.pck","rb")
    data = pickle.load(pickle_in)
    return data

In [ ]:
# Save your objects for future purposes 
save_object(Feature_Scaler , "Feature_Scaler")
save_object(Target_Scaler , "Target_Scaler")

Model Building

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint , ReduceLROnPlateau

save_best = ModelCheckpoint("best_weights.h5", monitor='val_loss', save_best_only=True, save_weights_only=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.25,patience=4, min_lr=0.00001,verbose = 1)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense , Dropout , LSTM , Bidirectional

model = Sequential() #keep track of all layers

model.add(Bidirectional(LSTM(512 ,return_sequences=True , recurrent_dropout=0.1, input_shape=(32, 2))))
model.add(LSTM(256 ,recurrent_dropout=0.1))
model.add(Dropout(0.3))
model.add(Dense(64 , activation='elu')) #elu is a modified version of leaky relu looks like relu and tanh activation
model.add(Dropout(0.3))
model.add(Dense(32 , activation='elu'))
model.add(Dense(1 , activation='linear'))

In [ ]:
#optimizer = tf.keras.optimizers.Adam(learning_rate=0.002)
optimizer = tf.keras.optimizers.SGD(learning_rate = 0.002) #sgd and adams are used and sgd is used because of batch size
model.compile(loss='mse', optimizer=optimizer)

In [ ]:
history = model.fit(Xtrain, Ytrain,
            epochs=10,
            batch_size = 1,
            verbose=1,
            shuffle=False ,
            validation_data=(Xtest , Ytest),
            callbacks=[reduce_lr , save_best])

Epoch 1/10
622/622 [==============================] - 710s 1s/step - loss: 0.0154 - val_loss: 0.4273
Epoch 2/10
622/622 [==============================] - 694s 1s/step - loss: 0.0115 - val_loss: 0.3912
Epoch 3/10
622/622 [==============================] - 692s 1s/step - loss: 0.0098 - val_loss: 0.3539
Epoch 4/10
622/622 [==============================] - 691s 1s/step - loss: 0.0087 - val_loss: 0.3304
Epoch 5/10
622/622 [==============================] - 685s 1s/step - loss: 0.0083 - val_loss: 0.3032
Epoch 6/10
622/622 [==============================] - 682s 1s/step - loss: 0.0082 - val_loss: 0.2984
Epoch 7/10
622/622 [==============================] - 681s 1s/step - loss: 0.0073 - val_loss: 0.2598
Epoch 8/10
622/622 [==============================] - 685s 1s/step - loss: 0.0064 - val_loss: 0.2392
Epoch 9/10
622/622 [==============================] - 684s 1s/step - loss: 0.0069 - val_loss: 0.2171
Epoch 10/10
622/622 [==============================] - 688s 1s/step - loss: 0.0063 - val_lo

In [ ]:
# Checking the model Structure 
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional (Bidirectional (1, 32, 1024)             2109440   
_________________________________________________________________
lstm_1 (LSTM)                (1, 256)                  1311744   
_________________________________________________________________
dropout (Dropout)            (1, 256)                  0         
_________________________________________________________________
dense (Dense)                (1, 64)                   16448     
_________________________________________________________________
dropout_1 (Dropout)          (1, 64)                   0         
_________________________________________________________________
dense_1 (Dense)              (1, 32)                   2080      
_________________________________________________________________
dense_2 (Dense)              (1, 1)                    3

In [ ]:
# Load the best weights
model.load_weights("best_weights.h5")

Visualize prediction on Test Set

In [ ]:
Predictions = model.predict(Xtest)

In [ ]:
Predictions = Target_Scaler.inverse_transform(Predictions)
Actual = Target_Scaler.inverse_transform(Ytest)

In [ ]:
Predictions = np.squeeze(Predictions , axis = 1)
Actual = np.squeeze(Actual , axis = 1)

In [ ]:
# Creating Sample Test Dataframe
test_dataframe_dict = {'Actual' : list(Actual) , 'Predicted' : list(Predictions)}
test_df = pd.DataFrame.from_dict(test_dataframe_dict)

test_df.index = data.index[-test_length:]

In [ ]:
test_df.head()

,Actual,Predicted
Date,,
2020-09-01,218.100006,202.730057
2020-09-02,216.250000,202.609833
2020-09-03,213.149994,203.731583
2020-09-04,206.600006,205.387711
2020-09-07,207.899994,206.347961


In [ ]:
# Check the trend in Volume Traded
fig = go.Figure()

fig.add_trace(go.Scatter(x = test_df.index , y = Actual , mode = 'lines' , name='Actual'))
fig.add_trace(go.Scatter(x = test_df.index , y = Predictions , mode = 'lines' , name='Predicted'))
fig.show()

Visualize Prediction on whole data

In [ ]:
Total_features = np.concatenate((Xtrain , Xtest) , axis = 0)

In [ ]:
Total_Targets = np.concatenate((Ytrain , Ytest) , axis = 0)

In [ ]:
Predictions = model.predict(Total_features)

In [ ]:
Predictions = Target_Scaler.inverse_transform(Predictions)
Actual = Target_Scaler.inverse_transform(Total_Targets)

In [ ]:
Predictions = np.squeeze(Predictions , axis = 1)
Actual = np.squeeze(Actual , axis = 1)

In [ ]:
# Check the trend in close Traded
fig = go.Figure()

fig.add_trace(go.Scatter(x = data.index , y = Actual , mode = 'lines' , name='Actual'))
fig.add_trace(go.Scatter(x = data.index , y = Predictions , mode = 'lines' , name='Predicted'))
fig.show()

In [ ]:
model.save("Model.h5")
loaded_model = tf.keras.models.load_model("Model.h5")

In [ ]:
def PredictStockPrice(Model , DataFrame , PreviousDate , feature_length = 32):
    idx_location = DataFrame.index.get_loc(PreviousDate)
    Features = DataFrame.iloc[idx_location - feature_length : idx_location,:].values
    Features = np.expand_dims(Features , axis = 0)
    Features = Feature_Scaler.transform(Features)
    Prediction = Model.predict(Features)
    Prediction = Target_Scaler.inverse_transform(Prediction)
    return Prediction[0][0]

In [ ]:
PredictStockPrice(loaded_model , data , '2020-12-09')

219.98428